# Imports


In [2]:
%cd drive/My\ Drive/3-Other_Research/Image_Schema_Annotations
!ls

[Errno 2] No such file or directory: 'drive/My Drive/3-Other_Research/Image_Schema_Annotations'
/content
sample_data  VUA_test_all.csv  VUA_train.csv


In [3]:
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.1 MB/s eta 0:00:00


In [4]:
#torch and tranformers for model and training
!pip install pynvml
import torch  
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import AdamW                            
from transformers import get_linear_schedule_with_warmup
from transformers import DataCollatorForTokenClassification
from transformers import Trainer, TrainingArguments
import sentencepiece

#sklearn for evaluation
from sklearn import preprocessing                       
from sklearn.metrics import classification_report        
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split


#utilities
import pandas as pd
import glob, os
import time
import datetime
import random
import numpy as np
import matplotlib.pyplot as plt
#% matplotlib inline
import seaborn as sns
import pickle         # for saving data structures
from pynvml import *  # for checking gpu memory
import re

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.6 MB/s eta 0:00:00


In [5]:
# connect to GPU 
device = torch.device('cuda')

print('Connected to GPU:', torch.cuda.get_device_name(0))

Connected to GPU: Tesla T4


# Prepare Data

**Functions for preprocessing and creating of Training Data**

In [6]:
model_name="xlm-roberta-base"  #base or large
random_validation=True
val_percentage=0.1

In [7]:
# load datasets
test_data=pd.read_csv("VUA_test_all.csv", engine="python")
train_data=pd.read_csv("VUA_train.csv", engine="python")

In [8]:
def format_for_TokenClf(df):
  data_list = []
  sentence=[]
  labels=[]
  for index, row in df.iterrows():
    if row["id"][-2:]=="_1" and index!=0:
      data_list.append((sentence, labels))
      sentence=[]
      labels=[]
    if row["label"]==1:
      label="m"
    else:
      label="l"
    sentence.append(str(row["word"]))
    labels.append(label)
    if index==len(df)-1:
      data_list.append((sentence, labels))

  return data_list

In [9]:
test_data=format_for_TokenClf(test_data)
train_data=format_for_TokenClf(train_data)
print(test_data[0])

(['The', 'Labour', 'Party', 'Conference', ':', 'Policy', 'review', 'throws', 'a', 'spanner', 'in', 'the', 'Whitehall', 'machinery'], ['l', 'l', 'l', 'l', 'l', 'l', 'l', 'm', 'l', 'm', 'm', 'l', 'l', 'm'])


In [10]:
train_data, val_data= train_test_split(train_data, shuffle=random_validation, test_size=val_percentage)

In [11]:
print("Trained sentences: ", len(train_data))
print("Val sentences: ", len(val_data))
print("Test sentences: ", len(test_data))

Trained sentences:  10898
Val sentences:  1211
Test sentences:  4080


In [12]:
#Train
train_tags=[tupl[1] for tupl in train_data]
train_texts=[tupl[0] for tupl in train_data]

#Val 
val_tags=[tupl[1] for tupl in val_data]
val_texts=[tupl[0] for tupl in val_data]

#Test
test_tags=[tupl[1] for tupl in test_data]
test_texts=[tupl[0] for tupl in test_data]

In [13]:
for txt in train_tags:
  if not isinstance(txt, list):
    print(txt)

In [14]:
print(test_texts[0])
print(test_tags[0])

['The', 'Labour', 'Party', 'Conference', ':', 'Policy', 'review', 'throws', 'a', 'spanner', 'in', 'the', 'Whitehall', 'machinery']
['l', 'l', 'l', 'l', 'l', 'l', 'l', 'm', 'l', 'm', 'm', 'l', 'l', 'm']


# Tokenize 

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [16]:
label_list=["l", "m"]
label_to_id = {l: i for i, l in enumerate(label_list)}
num_labels=len(label_list)

def tokenize_align_and_encode_labels(texts, tags):
  tokenized_inputs = tokenizer(
      texts,
      padding=True,
      truncation=True,
      is_split_into_words=True,
  )
  labels = []
  for i, label in enumerate(tags):
      word_ids = tokenized_inputs.word_ids(batch_index=i)
      previous_word_idx = None
      label_ids = []
      for word_idx in word_ids:
          if word_idx is None:
              label_ids.append(-100)
          elif word_idx != previous_word_idx:
              label_ids.append(label_to_id[label[word_idx]])
          else:
              label_ids.append(-100)
          previous_word_idx = word_idx

      labels.append(label_ids)
  tokenized_inputs["labels"] = labels
  return tokenized_inputs  

test_input_and_labels = tokenize_align_and_encode_labels(test_texts, test_tags)

val_input_and_labels = tokenize_align_and_encode_labels(val_texts, val_tags)

train_input_and_labels = tokenize_align_and_encode_labels(train_texts, train_tags)

In [17]:
# create dataset
class OurDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

test_dataset = OurDataset(test_input_and_labels, test_input_and_labels["labels"])

train_dataset = OurDataset(train_input_and_labels, train_input_and_labels["labels"])

val_dataset = OurDataset(val_input_and_labels, val_input_and_labels["labels"])


In [18]:
test_dataset.__getitem__(0)

{'input_ids': tensor([     0,    581,    239,  38648,  31016, 114732,    152,  80042,   8347,
         104250,      7,     10,  27734,   1679,     23,     70,  22392,  29907,
          36279,   1294,      2,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1,      1,      1,      1,      1,      1,      1,      1,      1,
              1

# Training

In [19]:

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    true_labels = [[label_list[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    true_labels = [val for sublist in true_labels for val in sublist]
    true_predictions = [val for sublist in true_predictions for val in sublist]
    print(classification_report(true_labels, true_predictions))
    precision, recall, f1, _ = precision_recall_fscore_support(true_labels, true_predictions, average="weighted")

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

In [20]:
# training arguments

training_args = TrainingArguments(
    output_dir='./MetaphorExtraction/results',          # output directory
    num_train_epochs=10,                                # total # of training epochs
    per_device_train_batch_size=8,                      # batch size per device during training
    per_device_eval_batch_size=16,                      # batch size for evaluation
    warmup_steps=0,                                     # number of warmup steps for learning rate scheduler
    weight_decay=0,                                     # strength of weight decay
    learning_rate=2e-5,
    logging_dir='./MetaphorExtraction/logs',            
    evaluation_strategy= "epoch",                       
    save_strategy = "epoch",
    load_best_model_at_end=True,                        
    metric_for_best_model="f1",
    greater_is_better=True
)

In [21]:
# initialize model
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=num_labels)


Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-st

In [22]:
nvmlInit()
h = nvmlDeviceGetHandleByIndex(0)
info = nvmlDeviceGetMemoryInfo(h)
print(f'Total  : {info.total}')
print(f'Free   : {info.free}')
print(f'Used   : {info.used}')

Total  : 16106127360
Free   : 15832252416
Used   : 273874944


In [23]:
# initialize huggingface trainer
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset = train_dataset, 
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

In [24]:
# train (record f1 all together = 0.961)
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.159800,0.129475,0.942351,0.945958,0.942662
2,0.122500,0.120000,0.952097,0.954104,0.952674
3,0.089900,0.124402,0.955686,0.955958,0.955817
4,0.069700,0.139890,0.955405,0.956913,0.955918
5,0.052400,0.159050,0.956042,0.957418,0.956534
6,0.035600,0.193099,0.956382,0.957137,0.956715
7,0.027300,0.200244,0.957464,0.958148,0.957768
8,0.019600,0.228921,0.957957,0.958823,0.958321
9,0.015200,0.264573,0.957030,0.958317,0.957496
10,0.011500,0.269248,0.957796,0.958710,0.958175


              precision    recall  f1-score   support

           l       0.96      0.98      0.97     15981
           m       0.81      0.62      0.70      1820

    accuracy                           0.95     17801
   macro avg       0.88      0.80      0.84     17801
weighted avg       0.94      0.95      0.94     17801

              precision    recall  f1-score   support

           l       0.97      0.98      0.97     15981
           m       0.82      0.71      0.76      1820

    accuracy                           0.95     17801
   macro avg       0.89      0.85      0.87     17801
weighted avg       0.95      0.95      0.95     17801

              precision    recall  f1-score   support

           l       0.97      0.98      0.98     15981
           m       0.79      0.78      0.78      1820

    accuracy                           0.96     17801
   macro avg       0.88      0.88      0.88     17801
weighted avg       0.96      0.96      0.96     17801

              preci

TrainOutput(global_step=13630, training_loss=0.06124713947382904, metrics={'train_runtime': 4434.7632, 'train_samples_per_second': 24.574, 'train_steps_per_second': 3.073, 'total_flos': 9010022517485280.0, 'train_loss': 0.06124713947382904, 'epoch': 10.0})

In [25]:
trainer.evaluate(val_dataset)

              precision    recall  f1-score   support

           l       0.97      0.98      0.98     15981
           m       0.82      0.77      0.79      1820

    accuracy                           0.96     17801
   macro avg       0.89      0.88      0.89     17801
weighted avg       0.96      0.96      0.96     17801



{'eval_loss': 0.22892069816589355,
 'eval_precision': 0.9579574875658847,
 'eval_recall': 0.9588225380596596,
 'eval_f1': 0.9583211559940139,
 'eval_runtime': 8.0429,
 'eval_samples_per_second': 150.568,
 'eval_steps_per_second': 9.449,
 'epoch': 10.0}

In [26]:
# Score on the test set
trainer.evaluate(test_dataset)

              precision    recall  f1-score   support

           l       0.96      0.98      0.97     51540
           m       0.81      0.71      0.76      6819

    accuracy                           0.95     58359
   macro avg       0.89      0.85      0.86     58359
weighted avg       0.94      0.95      0.95     58359



{'eval_loss': 0.27304166555404663,
 'eval_precision': 0.9449100810737393,
 'eval_recall': 0.9470175979711783,
 'eval_f1': 0.9455630536857341,
 'eval_runtime': 33.7552,
 'eval_samples_per_second': 120.87,
 'eval_steps_per_second': 7.554,
 'epoch': 10.0}

In [27]:
from datetime import date
trainer.save_model("./saved-models/metaphor_extraction_"+str(date.today())+"_randVal-"+str(random_validation)+"_"+model_name)

# Using the Model for Inference

In [28]:
from transformers import pipeline

In [29]:
label_list= ['literal',"metaphoric"]
label_dict_relations={ i : l for i, l in enumerate(label_list) }

In [32]:
PATH = "/content/saved-models/metaphor_extraction_2023-05-06_randVal-True_xlm-roberta-base"
model_metaphor_detection = AutoModelForTokenClassification.from_pretrained(PATH, id2label=label_dict_relations)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [33]:
pipeline_metaphors=pipeline("ner", model=model_metaphor_detection, tokenizer=tokenizer, aggregation_strategy="simple")

In [34]:
pipeline_metaphors("This young man knows how to climb the social ladder")

[{'entity_group': 'metaphoric',
  'score': 0.97492665,
  'word': 'This',
  'start': 0,
  'end': 4},
 {'entity_group': 'literal',
  'score': 0.9999875,
  'word': 'young man knows how to',
  'start': 5,
  'end': 27},
 {'entity_group': 'metaphoric',
  'score': 0.9992048,
  'word': 'climb',
  'start': 28,
  'end': 33},
 {'entity_group': 'literal',
  'score': 0.9999804,
  'word': 'the social',
  'start': 34,
  'end': 44},
 {'entity_group': 'metaphoric',
  'score': 0.99914396,
  'word': 'ladder',
  'start': 45,
  'end': 51}]